In [ ]:
%matplotlib nbagg
import matplotlib.pyplot as plt

In [ ]:
import os
import h5py
import hdf5plugin
import numpy

## Create 2D VDS over 3D .npy file

In [ ]:
import struct

def npy_to_h5dataset(npyfilename, group, name):
    """Create a HDF5 dataset to access a .npy file

    :param str npyfilename: Name of the .npy file.
        WARNING: This filename is used as is to reference the .npy file
        from the HDF5 file! Beware of absolute/relative path use.
    :param h5py.Group group: HDF5 group where to create the dataset
    :pram str name: Name of the HDF5 dataset
    """
    with open(npyfilename, mode='rb') as f:
        header = struct.unpack('6sBB', f.read(8))
        assert header == (b'\x93NUMPY', 1, 0)
        length = struct.unpack('<H', f.read(2))[0]
        f.seek(8)  # Seek back to end of magic header
        shape, fortran_order, dtype = numpy.lib.format.read_array_header_1_0(f)        
        assert not fortran_order

    # Offset in bytes of the array
    offset = length + 10
    # Size in bytes of the array
    size = numpy.prod(shape) * dtype.itemsize
    
    return group.create_dataset(
        name,
        shape=shape,
        dtype=dtype,
        external=[(os.path.abspath(npyfilename), offset, offset + size)])

In [ ]:
# Create HDF5 file a 3D external dataset stored in a .npy file and a 2D VDS mapping over it 
with h5py.File("/media/nvme/tvincent/testhdf5.h5", mode="w") as f:
    # Add stack of images
    images = npy_to_h5dataset('/media/nvme/tvincent/wasps_full.npy', f, 'images')

    # Prepare VDS layout
    assert images.ndim == 3
    nbimgs = int(numpy.sqrt(images.shape[0]))
    shape = nbimgs * images.shape[1], nbimgs * images.shape[2]

    layout = h5py.VirtualLayout(shape=shape, dtype=images.dtype)
    source = h5py.VirtualSource(images)
    for row in range(nbimgs):
        for col in range(nbimgs):
            layout[row*images.shape[1]:(row+1)*images.shape[1],
                   col*images.shape[2]:(col+1)*images.shape[2]] = source[row * nbimgs + col]

    # Create VDS
    f.create_virtual_dataset('level0', layout, fillvalue=0)

## Create binned levels

In [ ]:
def binning(image):
    assert image.ndim == 2
    return 0.25 * (image[:-1:2, :-1:2] + image[:-1:2, 1::2] + image[1::2, :-1:2] + image[1::2, 1::2])

def create_binned_level(group, name, previous, chunk=None):
    """Create a new HDF5 dataset with previous data binned 2x2

    :param h5py.Group group:
    :param str name:
    :param Union[h5py.Dataset,numpy.ndarray] previous:
    :param Union[None,List[int]] chunk:
        If not None run process by chunks.
        Chunk is defined in the output space.
    """
    if chunk is None:
        group[name] = binning(previous[()])
        return group[name]

    assert previous.ndim == 2
    shape = previous.shape[0] // 2, previous.shape[1] // 2
    dataset = group.create_dataset(
        name, shape=shape, dtype=previous.dtype)

    for row in range(shape[0] // chunk[0] + numpy.sign(shape[0] % chunk[0])):
        for col in range(shape[1] // chunk[1] + numpy.sign(shape[1] % chunk[1])):
            rbegin, rend = row * chunk[0], (row + 1) * chunk[0]
            cbegin, cend = col * chunk[1], (col + 1) * chunk[1]
            dataset[rbegin:rend, cbegin:cend] = binning(previous[rbegin*2:rend*2, cbegin*2:cend*2])

    return dataset

In [ ]:
with h5py.File("/media/nvme/tvincent/testhdf5.h5", mode="a") as f:
    for index in range(1, 8):
        print('Index', index)
        create_binned_level(f, 'level%s' % index, previous=f['level%s' % (index-1)], chunk=(1024, 1024))

In [ ]:
with h5py.File("/media/nvme/tvincent/testhdf5.h5", mode="r") as f:
    print(f['images'].shape, f['images'].dtype)
    print(f['level0'].shape, f['level0'].dtype)
    print(f['level7'].shape, f['level7'].dtype)
    #plt.imshow(f['images'][1000].astype('float32'))
    plt.imshow(f['level1'][:3445, :3445].astype('float32'))

## Generate histogram

For float16, generate the counts for each value (and clip to finite ones).
This also works for uint16.

For 32/64 bits, normal histogramming needed.

Store histogram as `counts` and `values` along with the data: `level0_histo/counts` and `level0_histo/values`.

In [ ]:
def get_float16_counts(data, chunk_shape=None):
    """Return counts of occurence of float16 values.

    This allows fast histograming of float16.

    :param Union[h5py.Dataset,numpy.ndarray] data:
    :param Union[None,List[int]] chunk_shape:
    :rtype: numpy.ndarray
    """
    # TODO support nD data and nD chunk_shape
    assert data.dtype == numpy.float16
    assert data.ndim >= 2

    if chunk_shape is None:
        chunk_shape = data.shape[:2]
    assert len(chunk_shape) == 2

    counts = numpy.zeros((2**16,), dtype=numpy.int64)

    for row in range(0, data.shape[0], chunk_shape[0]):
        for col in range(0, data.shape[1], chunk_shape[1]):
            chunk = data[row:row+chunk_shape[0], col:col+chunk_shape[1]]
            uint16_view = chunk.view(dtype=numpy.uint16)
            uint16_view.shape = -1
            counts = counts + numpy.bincount(uint16_view, minlength=counts.size)
    return counts


def get_finite_counts(*args, **kwargs):
    """Returns values and counts

    :return: (values, counts)
    :rtype: List[numpy.ndarray,numpy.ndarray]
    """
    # TODO values and finite_mask can be cached
    counts = get_float16_counts(*args, **kwargs)
    values = numpy.arange(2**16, dtype=numpy.uint16).view(dtype=numpy.float16)

    # Filter-out NaN and inf
    finite_mask = numpy.isfinite(values)
    values = values[finite_mask]
    counts = counts[finite_mask]

    # Sort returned values
    indices = numpy.argsort(values)
    return numpy.take(values, indices), numpy.take(counts, indices)


In [ ]:
%%time
# Populate levels
with h5py.File("/media/nvme/tvincent/testhdf5.h5", mode="a") as f:
    for level in range(6, -1, -1):
        level_name = 'level%d' % level
        print(level_name)
        group = f.create_group('%s_histo' % level_name)
        values, counts = get_finite_counts(f[level_name], chunk_shape=(2048, 2048))
        group['counts'] = counts
        group['values'] = values

In [ ]:
from collections import namedtuple

Stats = namedtuple('Stats', ('min_', 'max_', 'mean', 'std', 'sum_', 'counts'))

def get_stats_from_histo(values, counts):
    """Returns basic stats indicator computed from histogram

    :param numpy.ndarray values:
    :param numpy.ndarray counts:
    :rtype: Stats
    """
    subset = values[counts != 0]
    min_, max_ = subset[0], subset[-1]
    nbcounts = numpy.sum(counts, dtype=numpy.int64)
    sum_ = numpy.sum(values * counts, dtype=numpy.float64)
    mean = sum_ / nbcounts
    std = numpy.sqrt(numpy.sum(counts * (values.astype(numpy.float64) - mean)**2) / nbcounts)
    return Stats(
        min_=subset[0],
        max_=subset[-1],
        sum_=sum_,
        counts=nbcounts,
        mean=mean,
        std=std,
    )


In [ ]:
%%time

with h5py.File("/media/nvme/tvincent/testhdf5.h5", mode="r") as f:
    data = f['level4']
    values, counts = get_finite_counts(data)
    stats = get_stats_from_histo(values, counts)

stats

In [ ]:
%%time

with h5py.File("/media/nvme/tvincent/testhdf5.h5", mode="r") as f:
    data = f['level4'][()]

Stats(
        min_=numpy.min(data),
        max_=numpy.max(data),
        sum_=numpy.sum(data, dtype=numpy.float64),
        counts=data.size,
        mean=numpy.mean(data, dtype=numpy.float64),
        std=numpy.std(data, dtype=numpy.float64),
    )

## Update pyramid in live

In [ ]:
def binning(image):
    assert image.ndim == 2
    return 0.25 * (image[:-1:2, :-1:2] + image[:-1:2, 1::2] + image[1::2, :-1:2] + image[1::2, 1::2])

def update_binned_levels(levels, previous, rbegin, rend, cbegin, cend):
    """Update sub-region of a pyramid of image

    :param List[Union[h5py.Dataset,numpy.ndarray]] levels:
        Iterable of pyramid levels to update from largest to smallest.
    :param Union[h5py.Dataset,numpy.ndarray] previous:
        Updated level which is the one before the first one of levels.
    :param int rbegin:
    :param int rend:
    :param int cbegin:
    :param int cend:
    """
    for level in levels:
        print(level)
        # Start and end on even index, eventually enlarging the area 
        if rbegin % 2 != 0:
            rbegin -= 1
        if rend % 2 != 0:
            rend += 1
        if cbegin % 2 != 0:
            cbegin -= 1
        if cend % 2 != 0:
            cend += 1

        # TODO maybe fails on the edges
        level[rbegin//2:rend//2, cbegin//2:cend//2] = binning(previous[rbegin:rend, cbegin:cend])
        rbegin //= 2
        rend //= 2
        cbegin //= 2
        cend //= 2
        previous = level


In [ ]:
%%time

# Faster for smaller chunks, what else

with h5py.File("/media/nvme/tvincent/testhdf5.h5", mode="a") as f:
    for index in range(4):
        rbegin, rend = 2048*index, 2048*(index+1)
        cbegin, cend = 2048*index, 2048*(index+1)

        update_binned_levels(
            levels=[f['level%d' % i] for i in range(1, 8)],
            previous=f['level0'],
            rbegin=rbegin,
            rend=rend,
            cbegin=cbegin,
            cend=cend)


In [ ]:

# Raw image with NaN or 0
# Fill level0 over time
# Update other levels from level0
# Notify?


# Data/colormap update:
# Store a histogram: edges, count for each level and use it for autoscale colormap.
#   - Which number of bins?
#   - float16 specific histo using count of each possible value
#     (easy generation by using binary representation as index)
# Send DATA event each time displayed level change? to update colormap/histo/profile?

In [ ]:
from h5glance import H5Glance
H5Glance("/media/nvme/tvincent/testhdf5.h5")

In [ ]:
# Get data info
with h5py.File("/media/nvme/tvincent/testhdf5.h5", mode="r") as f:
    dataset = f['level0']
    shape, dtype = dataset.shape, dataset.dtype

# Create empty file
with h5py.File("/media/nvme/tvincent/live_update.h5", mode="w") as f:
    for index in range(0, 8):
        print('Index', index, shape, dtype)
        value = numpy.nan if dtype.kind == 'f' else 0
        dataset = f.create_dataset(
            'level%s' % index, shape=shape, dtype=dtype, fillvalue=value)
        shape = shape[0] // 2, shape[1] // 2

In [ ]:
import time

chunk_shape = 1024, 1024

# Get data shape
with h5py.File("/media/nvme/tvincent/live_update.h5", mode="r") as f:
    shape = f['level0'].shape

for rbegin in range(0, shape[0], chunk_shape[0]):
    for cbegin in range(1, shape[1], chunk_shape[1]):
        rend = rbegin + chunk_shape[0]
        cend = cbegin + chunk_shape[1]

        # Read data
        with h5py.File("/media/nvme/tvincent/testhdf5.h5", mode="r") as f:
            data = f['level0'][rbegin:rend, cbegin:cend]

        # Update pyramid file
        with h5py.File("/media/nvme/tvincent/live_update.h5", mode="a") as f:
            f['level0'][rbegin:rend, cbegin:cend] = data
            update_binned_levels(
                levels=[f['level%d' % i] for i in range(1, 8)],
                previous=f['level0'],
                rbegin=rbegin,
                rend=rend,
                cbegin=cbegin,
                cend=cend)

        # TODO update histogram
